In [1]:
import pandas as pd
import numpy as np
import requests
from requests import get
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup # for web scraping
import seaborn as sns # for beautiful graphs
import scipy.stats as stats # to calculate r^2 for linear regressions
from scipy.stats import powerlaw # for plotting linear regressions
import statsmodels as sm
import matplotlib.ticker as mtick
import re
sns.set()

# Dataset Cleaning 

First , we need to remove the empty columns from the dataset.

In [2]:
#opening the data
data= pd.read_csv('/Users/Mariam/Desktop/chicago-food-inspections/food-inspections.csv',delimiter=',')

# drop all the empty columns
data.drop(['Historical Wards 2003-2015', 'Zip Codes', 'Community Areas','Census Tracts','Wards'], axis=1,inplace=True)

#show the dataframe
display(data.head(3))

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2320315,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,2216009.0,Daycare Above and Under 2 Years,Risk 1 (High),1300 W 99TH ST,CHICAGO,IL,60643.0,2019-10-23T00:00:00.000,License Re-Inspection,Pass,NaN,41.714168,-87.655291,"{'longitude': '41.7141680989703', 'latitude': ..."
1,2320342,YOLK TEST KITCHEN,YOLK TEST KITCHEN,2589655.0,Restaurant,Risk 1 (High),1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,2019-10-23T00:00:00.000,Canvass,Pass w/ Conditions,23. PROPER DATE MARKING AND DISPOSITION - Comm...,41.913588,-87.682203,"{'longitude': '41.9135877900482', 'latitude': ..."
2,2320328,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,2583309.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,2019-10-23T00:00:00.000,Canvass,Out of Business,NaN,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':..."


We need to standardize columns format in order to make it more friendly to use.

In [3]:
#We create a function that fills empty space by '_' and lower case all the letters (reformat all column headers)
def standardize(column):
    column = column.lower().replace(" ", "_")
    column = re.sub('\W+',"", column)
    if len(column) > 1:
        if column[-1] == "_":
            return column[:-1]
    return column

#application of the function to the dataset
data.columns = [standardize(x) for x in data.columns]
display(data.columns)


Index(['inspection_id', 'dba_name', 'aka_name', 'license', 'facility_type',
       'risk', 'address', 'city', 'state', 'zip', 'inspection_date',
       'inspection_type', 'results', 'violations', 'latitude', 'longitude',
       'location'],
      dtype='object')

We need to check if the inspection ID is unique. If it is not the case ,we need to remove the duplicates as an ID reffers to an unique inspection.

In [4]:
#Check is the inspection ID is unique
display(data['inspection_id'].is_unique)

False

In [5]:
#Removing the duplicates 
data.drop_duplicates('inspection_id', inplace=True)

#check if all the duplicates were removed 
display(data['inspection_id'].is_unique)

True

We need to remove all the NA values since we can't use this kind of information. But we need to remove them only from particular column. ( ex : for the violations , NA only means that there were no violations so we need to keep this NA).

In [6]:
#Remove NA values from relevent columns
data.dropna(subset=['inspection_date','license','latitude','longitude'],inplace=True)

We need to remove the time of inspection (useless information) in order to clean the inspection date column

In [7]:
#We remove the time ( all the caracters after 'T')
data['inspection_date']=data['inspection_date'].apply(lambda x : x.split('T')[0])
display(data.head(3))

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2320315,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,2216009.0,Daycare Above and Under 2 Years,Risk 1 (High),1300 W 99TH ST,CHICAGO,IL,60643.0,2019-10-23,License Re-Inspection,Pass,NaN,41.714168,-87.655291,"{'longitude': '41.7141680989703', 'latitude': ..."
1,2320342,YOLK TEST KITCHEN,YOLK TEST KITCHEN,2589655.0,Restaurant,Risk 1 (High),1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,2019-10-23,Canvass,Pass w/ Conditions,23. PROPER DATE MARKING AND DISPOSITION - Comm...,41.913588,-87.682203,"{'longitude': '41.9135877900482', 'latitude': ..."
2,2320328,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,2583309.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,2019-10-23,Canvass,Out of Business,NaN,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':..."


We need to check if there are only information from Chicago.

In [8]:
#check if we have only data from chicage, if not we need to remove all the extra information
data.city.unique()

array(['CHICAGO', nan, 'Chicago', 'CCHICAGO', 'CHICAGO.',
       'CHESTNUT STREET', 'CHICAGOCHICAGO', 'chicago', 'CHICAGOHICAGO',
       'CHicago', '312CHICAGO', 'BEDFORD PARK', 'CHCICAGO',
       'CHARLES A HAYES', 'CHCHICAGO', 'CHICAGOI', 'SUMMIT', 'WESTMONT',
       'LOMBARD', 'INACTIVE', 'alsip', 'BLUE ISLAND'], dtype=object)

We need to remove :   Bedford Park (Gas Station) ; Blue Island; Lombard ( a village near to Chicago) , Summit ( a city near to Chicago) ; WESTMONT ( village near Chicago) ; aslip (suburb of chicago)
We need to replace by chicago : 'CHARLES A HAYES'  (postal location), 312Chicago (Restaurant) ; CHICAGOI (Chicago)  ; CHESTNUT STREET (street in chicago); INACTIVE (out of business restaurant in Chicago); Chestnut street ;

In [9]:
#Check if the state is unique
display(data.state.unique())

#As the state is unique and we will not use this columns for our further investigations, we can drop it 
data.drop(['state'], axis=1,inplace=True)

#Check if there are other city than Chicago
display(data.inspection_id.groupby(data['city']).count())

#Drop the selected locations 
data = data[~data['city'].isin(["BEDFORD PARK", "BLUE ISLAND", "LOMBARD","SUMMIT","WESTMONT","alsip"])]

#check if the column is clean 
display(data.inspection_id.groupby(data['city']).count())

#now that we are sure that we have only information from Chicago ,we can delete the city columns
data.drop(['city'], axis=1,inplace=True)

array(['IL', nan], dtype=object)

city
312CHICAGO              2
BEDFORD PARK            2
BLUE ISLAND             1
CCHICAGO               45
CHARLES A HAYES         4
CHCHICAGO               6
CHCICAGO                3
CHESTNUT STREET        11
CHICAGO            193192
CHICAGO.                2
CHICAGOCHICAGO          7
CHICAGOHICAGO           2
CHICAGOI                3
CHicago                12
Chicago               317
INACTIVE                8
LOMBARD                 1
SUMMIT                  4
WESTMONT                1
alsip                   1
chicago                82
Name: inspection_id, dtype: int64

city
312CHICAGO              2
CCHICAGO               45
CHARLES A HAYES         4
CHCHICAGO               6
CHCICAGO                3
CHESTNUT STREET        11
CHICAGO            193192
CHICAGO.                2
CHICAGOCHICAGO          7
CHICAGOHICAGO           2
CHICAGOI                3
CHicago                12
Chicago               317
INACTIVE                8
chicago                82
Name: inspection_id, dtype: int64

If we explore the license numbers, we find that there are some null license number. We need to remove them,

In [10]:
# Drop "0.0" licenses
data = data[data.license != 0.0]

# Score creation 

We want to make the violation column more readible.
Using the followingl link https://webapps1.chicago.gov/healthinspection/Code_Violations.jsp#minor  , each number is associated with a unique violation.
For simplification purpose, we will consider only the violations from 1 to 44 and the violation number 70. 

In [12]:
#function that split the violations number from the comments

def violation_separator(violations):
    violation_number = pd.Series([])   #creating an empty dataframe in order to stock the violation numbers
    if type(violations) == str:
        violations = violations.split(' | ') #each different violation is separated by a ' | ' in a dataframe cell
        for violation in violations:        #now, we can iterate on the differente violations of each inspection
            index = "#" + violation.split('.')[0]  #the index refers to the violation number
            violation_number[index] = 1 #add 1 if there is a violation #.. and 0 if not.
    return violation_number

#we create a new data frame by applying the function to the dataset thhe old one and fill the nan value by 0 . 1= violation , 0= no violation
violations_data = data.violations.apply(violation_separator).fillna(0)

In [13]:
# Generate column names
critical = [("#" + str(num)) for num in range(1, 15)]
serious = [("#" + str(num)) for num in range(15, 30)]
minor = [("#" + str(num)) for num in range(30, 45)]
minor.append("#70")

# Create complete list of column names
columns = critical + serious + minor

# Create dataframe using column names, violation data and inspection ID
violations_data = pd.DataFrame(violations_data, columns=columns)
violations_data['inspection_id'] = data.inspection_id
violations_data['license'] = data.license

In [23]:
#counting the kind of violation per inspection ID
violation_counts = pd.DataFrame({
    "critical_count": violations_data[critical].sum(axis=1),
    "serious_count": violations_data[serious].sum(axis=1),
    "minor_count": violations_data[minor].sum(axis=1)
})

violation_counts['inspection_id'] = data.inspection_id
violation_counts['license'] = data.license
# Display selection of sums dataframe
violation_counts.iloc[3:6]

,critical_count,serious_count,minor_count,inspection_id,license
3,0.0,0.0,0.0,2320319,2694702.0
4,2.0,0.0,0.0,2320228,2678250.0
5,0.0,2.0,1.0,2320261,2616198.0


In [24]:
#minor violatios are more frequent .
violation_counts.sum()

critical_count    5.912100e+04
serious_count     5.955400e+04
minor_count       4.373610e+05
inspection_id     2.785664e+11
license           3.096339e+11
dtype: float64

Here, we want to create a basic score that depends on the violations type (critical, serious or minor).
critical = 3 points
serious= 2 points
minor=1 points 
The lower the score, the better.

In [25]:
#calculation of the basic score for each inspection
basic_score=violation_counts.apply(lambda x: x.critical_count*3+x.serious_count*2+x.minor_count*1, axis = 1)
violation_counts['basic_score'] = basic_score
violation_counts

,critical_count,serious_count,minor_count,inspection_id,license,basic_score
0,0.0,0.0,0.0,2320315,2216009.0,0.0
1,0.0,2.0,1.0,2320342,2589655.0,5.0
2,0.0,0.0,0.0,2320328,2583309.0,0.0
3,0.0,0.0,0.0,2320319,2694702.0,0.0
4,2.0,0.0,0.0,2320228,2678250.0,6.0
...,...,...,...,...,...,...
194808,0.0,0.0,0.0,67732,1992039.0,0.0
194809,0.0,0.0,0.0,52234,2013328.0,0.0
194811,0.0,0.0,0.0,67733,1992040.0,0.0
194812,0.0,0.0,0.0,67757,1380279.0,0.0


In [26]:
#computation of the basic score of a facility
violation_counts.groupby('license')['basic_score'].sum()

license
1.0           0.0
2.0          65.0
9.0          90.0
40.0         81.0
43.0          0.0
             ... 
3846104.0    17.0
3846211.0     6.0
5852857.0     0.0
8700606.0     5.0
9999999.0     0.0
Name: basic_score, Length: 37031, dtype: float64

In [27]:
#check if the score computation is correct
violation_counts.loc[violation_counts['license']==40.0]


,critical_count,serious_count,minor_count,inspection_id,license,basic_score
18725,3.0,0.0,1.0,2222357,40.0,10.0
27018,0.0,0.0,3.0,2151032,40.0,3.0
38448,0.0,0.0,2.0,2079140,40.0,2.0
39114,1.0,1.0,2.0,2078535,40.0,7.0
39535,1.0,1.0,4.0,2072109,40.0,9.0
48734,0.0,0.0,3.0,1995576,40.0,3.0
72422,0.0,0.0,5.0,1689246,40.0,5.0
89754,0.0,0.0,4.0,1418988,40.0,4.0
95410,0.0,0.0,2.0,1522306,40.0,2.0
108892,0.0,0.0,2.0,1372583,40.0,2.0


Problem of this way of scoring : the oldest facilities will have a higher score due to a higher number of investigation.
We need to find a better score that take into account this factor.
May be we should take only the last inspection results ? 